In [1]:
from pyspark import SparkContext

In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark import SparkContext

In [5]:
from pyspark.sql.functions import trim

In [6]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer, VectorIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import avg

In [7]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
from pyspark.sql.functions import trim

In [9]:
from pyspark.sql.functions import *

In [10]:
from pyspark.ml import Pipeline

In [11]:
from pyspark.ml.tuning import ParamGridBuilder 
from pyspark.ml.tuning import CrossValidator

In [12]:
from pyspark.ml.regression import RandomForestRegressor

In [13]:
from pyspark.ml import PipelineModel

In [14]:
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import OneHotEncoder, StringIndexer, StandardScaler, Imputer, VectorAssembler, SQLTransformer
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.linalg import Vectors

In [15]:
from pyspark.ml.classification import GBTClassifier 
from pyspark.ml.classification import RandomForestClassifier 
from pyspark.ml.evaluation import BinaryClassificationEvaluator 
import matplotlib.pyplot as plt
from pyspark.ml.feature import StandardScaler

In [16]:
from pyspark.ml.regression import LinearRegression

In [17]:
from sklearn.metrics import f1_score

In [18]:
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler

In [19]:
spark = SparkSession.builder.appName('wine-rf_model')\
                            .getOrCreate()

In [20]:
def return_parsed_df(path):
    temp_df = spark.read.csv(path,header='true', inferSchema='true', sep=';')
    temp_df_pd = temp_df.toPandas()
    temp_df_pd.columns = temp_df_pd.columns.str.strip('""')
    return spark.createDataFrame(temp_df_pd)

In [21]:
wine_train_df = return_parsed_df('TrainingDataset.csv')

In [22]:
wine_test_df = return_parsed_df('ValidationDataset.csv')

In [23]:
wine_train_df.columns

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [24]:
pd.DataFrame(wine_train_df.take(5), columns = wine_train_df.columns)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.9,0.22,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,6
1,7.6,0.39,0.31,2.3,0.082,23.0,71.0,0.9982,3.52,0.65,9.7,5
2,7.9,0.43,0.21,1.6,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5
3,8.5,0.49,0.11,2.3,0.084,9.0,67.0,0.9968,3.17,0.53,9.4,5
4,6.9,0.40,0.14,2.4,0.085,21.0,40.0,0.9968,3.43,0.63,9.7,6


In [25]:
wine_train_df = wine_train_df.dropna()

In [26]:
wine_test_df = wine_test_df.dropna()

In [27]:
all_col_exc_quality = [c for c in wine_train_df.columns if c != 'quality']

In [28]:
stages = []

In [29]:
label_stringIdx = StringIndexer(inputCol = 'quality', outputCol = 'label')
stages += [label_stringIdx]

In [30]:
assembler = VectorAssembler(inputCols=all_col_exc_quality, 
                            outputCol="features")

In [31]:
stages += [assembler]

In [33]:
pipeline = Pipeline(stages = stages)

In [34]:
pipelineModel = pipeline.fit(wine_train_df)

In [35]:
pd.DataFrame(wine_train_df.take(5), columns = wine_train_df.columns)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.9,0.22,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,6
1,7.6,0.39,0.31,2.3,0.082,23.0,71.0,0.9982,3.52,0.65,9.7,5
2,7.9,0.43,0.21,1.6,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5
3,8.5,0.49,0.11,2.3,0.084,9.0,67.0,0.9968,3.17,0.53,9.4,5
4,6.9,0.40,0.14,2.4,0.085,21.0,40.0,0.9968,3.43,0.63,9.7,6


In [36]:
dataDF = pipelineModel.transform(wine_train_df)
dataDF.printSchema()

root
 |-- fixed acidity: double (nullable = true)
 |-- volatile acidity: double (nullable = true)
 |-- citric acid: double (nullable = true)
 |-- residual sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free sulfur dioxide: double (nullable = true)
 |-- total sulfur dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: long (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [37]:
Data_test_df = pipelineModel.transform(wine_test_df)

In [38]:
Data_test_df.limit(3).toPandas().head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label,features
0,7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5,0.0,"[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978..."
1,7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5,0.0,"[7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.996..."
2,7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5,0.0,"[7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.99..."


In [39]:
dataDF.limit(3).toPandas().head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label,features
0,8.9,0.22,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,6,1.0,"[8.9, 0.22, 0.48, 1.8, 0.077, 29.0, 60.0, 0.99..."
1,7.6,0.39,0.31,2.3,0.082,23.0,71.0,0.9982,3.52,0.65,9.7,5,0.0,"[7.6, 0.39, 0.31, 2.3, 0.082, 23.0, 71.0, 0.99..."
2,7.9,0.43,0.21,1.6,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5,0.0,"[7.9, 0.43, 0.21, 1.6, 0.106, 10.0, 37.0, 0.99..."


In [40]:
selectedCols = ['label', 'features'] + wine_train_df.columns

In [41]:
Data_test_df = Data_test_df.select(selectedCols)

In [42]:
dataDF = dataDF.select(selectedCols)

In [43]:
pd.DataFrame(dataDF.take(5), columns = dataDF.columns)

,label,features,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,1.0,"[8.9, 0.22, 0.48, 1.8, 0.077, 29.0, 60.0, 0.99...",8.9,0.22,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,6
1,0.0,"[7.6, 0.39, 0.31, 2.3, 0.082, 23.0, 71.0, 0.99...",7.6,0.39,0.31,2.3,0.082,23.0,71.0,0.9982,3.52,0.65,9.7,5
2,0.0,"[7.9, 0.43, 0.21, 1.6, 0.106, 10.0, 37.0, 0.99...",7.9,0.43,0.21,1.6,0.106,10.0,37.0,0.9966,3.17,0.91,9.5,5
3,0.0,"[8.5, 0.49, 0.11, 2.3, 0.084, 9.0, 67.0, 0.996...",8.5,0.49,0.11,2.3,0.084,9.0,67.0,0.9968,3.17,0.53,9.4,5
4,1.0,"[6.9, 0.4, 0.14, 2.4, 0.085, 21.0, 40.0, 0.996...",6.9,0.40,0.14,2.4,0.085,21.0,40.0,0.9968,3.43,0.63,9.7,6


In [44]:
pd.DataFrame(Data_test_df.take(5), columns = Data_test_df.columns)

,label,features,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.0,"[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978...",7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5
1,0.0,"[7.8, 0.88, 0.0, 2.6, 0.098, 25.0, 67.0, 0.996...",7.8,0.88,0.00,2.6,0.098,25,67,0.9968,3.20,0.68,9.8,5
2,0.0,"[7.8, 0.76, 0.04, 2.3, 0.092, 15.0, 54.0, 0.99...",7.8,0.76,0.04,2.3,0.092,15,54,0.9970,3.26,0.65,9.8,5
3,1.0,"[11.2, 0.28, 0.56, 1.9, 0.075, 17.0, 60.0, 0.9...",11.2,0.28,0.56,1.9,0.075,17,60,0.9980,3.16,0.58,9.8,6
4,0.0,"[7.4, 0.7, 0.0, 1.9, 0.076, 11.0, 34.0, 0.9978...",7.4,0.70,0.00,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,5


In [45]:
print("Training Dataset Count: " + str(dataDF.count()))

Training Dataset Count: 1279


In [46]:
print("Test Dataset Count: " + str(Data_test_df.count()))

Test Dataset Count: 160


In [47]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')

In [48]:
rf_model = rf.fit(dataDF)

In [49]:
predictions = rf_model.transform(Data_test_df)

In [50]:
predictions.select('rawPrediction', 'prediction', 'probability').show(10)

+--------------------+----------+--------------------+
|       rawPrediction|prediction|         probability|
+--------------------+----------+--------------------+
|[12.8419385929427...|       0.0|[0.64209692964714...|
|[13.3516278954035...|       0.0|[0.66758139477017...|
|[12.3972632114059...|       0.0|[0.61986316057029...|
|[10.7431839731887...|       0.0|[0.53715919865943...|
|[12.8419385929427...|       0.0|[0.64209692964714...|
|[13.2258501062660...|       0.0|[0.66129250531330...|
|[14.0101453415581...|       0.0|[0.70050726707790...|
|[13.5373650770311...|       0.0|[0.67686825385155...|
|[13.0751722700496...|       0.0|[0.65375861350248...|
|[13.0626589647387...|       0.0|[0.65313294823693...|
+--------------------+----------+--------------------+
only showing top 10 rows



In [51]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [52]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

In [53]:
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.45


In [54]:
print("Test Area Under ROC: " +
str(evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})))

Test Area Under ROC: 0.55


In [55]:
stages += [rf]

In [56]:
pipeline = Pipeline(stages = stages)

In [57]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
params = ParamGridBuilder().build()
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=params,
                    evaluator=evaluator,
                    numFolds=5)

In [58]:
# dataDF = train data
# Data_test_df = test data
cvModel = cv.fit(wine_train_df)
predictions = cvModel.transform(wine_test_df)
predictions_pandas = predictions.toPandas()
print('Test Area Under PR: ', evaluator.evaluate(predictions))


Test Area Under PR:  0.55


In [59]:
cvModel.write().overwrite().save('output/rf_wine.model')

In [60]:
f1 = f1_score(predictions_pandas.label, predictions_pandas.prediction, average='weighted')

In [61]:
print(f1)

0.5217990121580548
